## Setup

In [ ]:
from specific import *

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

### Retrieve previous results from the 'model' notebook

In [ ]:
results, rf = cross_val_cache.load()

### Predict BA

In [ ]:
n_threads = 32
rf.n_jobs = n_threads
with parallel_backend("threading", n_jobs=n_threads):
    predicted_ba = get_masked_array(rf.predict(exog_data), master_mask)
target_ba = get_masked_array(endog_data.values, master_mask)

In [ ]:
ba_predicted = predicted_ba
ba_data = target_ba

model_name = "RF1"

# Plotting params.
figsize = (5, 3.33)
mpl.rcParams["figure.figsize"] = figsize

vmin = min((np.min(ba_predicted), np.min(ba_data)))
vmax = max((np.max(ba_predicted), np.max(ba_data)))

boundaries = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
coast_linewidth = 0.5
date_str = "2010-01 to 2015-01"

# Plotting predicted.
fig = cube_plotting(
    ba_predicted,
    cmap="brewer_RdYlBu_11_r",
    title=f"Predicted BA\n{date_str}",
    log=True,
    extend="min",
    boundaries=boundaries,
    colorbar_kwargs={"label": "Burned Area Fraction"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"predicted_ba_{model_name}", sub_directory="predictions")

# Plotting observed.
fig = cube_plotting(
    ba_data,
    cmap="brewer_RdYlBu_11_r",
    title=f"Predicted BA\n{date_str}",
    log=True,
    extend="min",
    boundaries=boundaries,
    colorbar_kwargs={"label": "Burned Area Fraction"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"observed_ba_{model_name}", sub_directory="predictions")

# Plotting differences.
diff_boundaries = [-1e3, -1e0, -1e-1, -1e-2, 0, 1e-1, 5e-1, 1e0]

frac_diffs = (ba_data - ba_predicted) / ba_data

fig = cube_plotting(
    frac_diffs,
    cmap="brewer_RdYlBu_11_r",
    title=f"BA Discrepancy\n{date_str}",
    log=True,
    boundaries=diff_boundaries,
    extend="min" if np.max(frac_diffs) <= max(diff_boundaries) else "both",
    colorbar_kwargs={"label": "(Observed - Predicted) / Observed"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"difference_ba_{model_name}", sub_directory="predictions"
)